In [1]:
# Importing dependencies

import json
import pandas as pd
import numpy as np

In [2]:
# using an fstring to open a file directory

file_dir = '/D/GitHubRepos/Movies-ETL'
new_file_dir = './'

# pd.read_json method only works for data thats already clean
# JSON data has every field filled in every time it is returned, called “flat data"

# loading the messy json file with "with" and json.load() to save

with open(f'{new_file_dir}/wikipedia.movies.json', mode = 'r') as file:
    messy_wiki_movies = json.load(file)

# loaded movies json becomes a list of dictionaries

len(messy_wiki_movies)

7311

In [3]:
# accessing dict records using slicing

messy_wiki_movies[:1]

# messy_wiki_movies[3500:3503]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [4]:
# importing kaggle to csv read

kaggle_metadata = pd.read_csv(f'{new_file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{new_file_dir}ratings.csv')

In [5]:
ratings.head(5)

ratings.describe()

ratings.dtypes

# .sample(n='number') shows any random rows
# good practice to check the last few rows and a random sample of rows
# for consistent structure and explicit, unique id for each data point

ratings.sample(n=4)

,userId,movieId,rating,timestamp
4686412,48132,2428,0.5,1366800341
6780335,69949,8464,5.0,1090977507
17298807,179633,293,4.5,1251188001
10282504,106124,908,5.0,1437055431


In [6]:
kaggle_metadata.head(5).tail(4)

kaggle_metadata.dtypes

# ratings.sample(n=9)

# list(kaggle_metadata.coloumns())

kaggle_metadata.dtypes
kaggle_metadata.sample(n=3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
44353,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://thenorthlander.com,412280,tt5059782,en,The Northlander,"In the year 2961, the time is after humanity a...",...,2016-10-14,0.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In 2961: The only rule is instinct,The Northlander,False,5.3,3.0
18725,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",http://www.blackballedthemovie.com,14346,tt0379184,en,Blackballed: The Bobby Dukes Story,"Caught cheating and banned for ten years, Bobb...",...,2004-01-01,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Second place is first place for losers,Blackballed: The Bobby Dukes Story,False,5.5,5.0
35964,False,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,57845,tt0107505,de,Mandroid,"In his hidden laboratory deep in Russia, Dr. K...",...,1993-08-18,0.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's Power is in Your Hands!,Mandroid,False,4.6,6.0


In [7]:
# INSPECT and identify a problem , PLAN and decide whether to fix it, EXECUTE data repair

# deleting obviously bad data, removing superfluous columns (with one value or missing an overwhelming amount of data)
# removing duplicate rows, consolidating columns, and reshaping the data

# articulating the problems clearly, devising a plan to modify the data and fix the problem
# even if it might affect one data point, can it affect other rows

# Modifying data structure includes pivoting the values of one column into multiple columns, aggregating rows,
# and merging multiple data sets

In [8]:
# inspecting wiki data

wiki_movies_df = pd.DataFrame(messy_wiki_movies)

wiki_movies_df.head(5)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# converting the absurd amount of coloumn names into a list

wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [18]:
# list comprehensions: rewind
# [expression for element in source_list]
# now we can [expression for element in source_list if filter_expression]

# filtering out wiki movies if their is a "director" or "directed by" key and a "imbd link" key

wiki_filterz = [movie for movie in messy_wiki_movies if ('Director' in movie or 'Directed by' in movie) and 'imdb_link' in movie]

len(wiki_filterz)

# # len(messy_wiki_movies)

wiki_director_imbd_filter_df = pd.DataFrame(wiki_filterz)

wiki_director_imbd_filter_df.head()


# down to 78 columns

wiki_director_imbd_filter_df.columns.tolist()

# adding to new df

wiki_filtered_movies_df = pd.DataFrame(wiki_director_imbd_filter_df)

wiki_filtered_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# filtering out episodes since we only want movies 

wiki_episodes = [movie for movie in messy_wiki_movies
                if ('Director' in movie or 'Directed by' in movie)
                 and 'imdb_link' in movie
                 and 'No. of episodes' not in movie]

len(wiki_episodes)

wiki_movies_dfN = pd.DataFrame(wiki_episodes)

wiki_movies_dfN.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# defining a function: Name, Parameters, Code block, Return value
# have to make sure dicts, lists data points dont get changed while calling on them inside a function
# thus, make copies using constructors like list(movies_list) and dict(movies_dict)

# enter lambda functions: anonymous functions that return a value without return call

In [32]:
# looking at the all the columns from the movies_df

wiki_movies_dfN.columns.tolist()

# alternate names in Arabic

wiki_movies_dfN[wiki_movies_dfN['Arabic'].notnull()]

wiki_movies_dfN[wiki_movies_dfN['Arabic'].notnull()]['url']

# sorting all the columns

sorted(wiki_movies_dfN.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplifie

In [39]:
# trying to copy movies for a non-destrictive copy (as to not edit out original list/dict values)


# def clean_movie(movie):
#     movie_copy = dict(movie)
    
#     def clean_movie(movie):
#         movie = dict(movie)
        
#         return movie


# calling a functions and have 'yard' as a parameter

def clean_movies(yard):
# since the movies are dicts and needs a non-destructive copy

#     creating a non-destrcutive copy
    movie = dict(yard)

# dict holds empty alternate titles
    alt_titles = {}
    
#     looping through all the alt titles
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        
#       if key exists in the movie object
        if key in movie:
        
#         add key:value pair to alt_titles and remove from movie
            alt_titles[key] = movie[key]
            movie.pop(key)

#         adding alt_title to movie object
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
            

    return movie

clean_movies = [clean_movies(movie) for movie in wiki_episodes]

wiki_movies_df = pd.DataFrame(clean_movies)

sorted(wiki_movies_df.columns.tolist())

# for some reason, canvas has hebrew and mccunne-reischaur added this list



['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']